In [1]:
$(crc console --credentials | sed "s/^.*\ '//" | sed "s/'.*$//" | tail -1)

Login successful.

You have access to 67 projects, the list has been suppressed. You can list all projects with 'oc projects'

Using project "imagetragick".


In [32]:
crc console --credentials

To login as a regular user, run 'oc login -u developer -p developer https://api.crc.testing:6443'.
To login as an admin, run 'oc login -u kubeadmin -p qGRkW-r8efv-M9JNo-9BkQn https://api.crc.testing:6443'


In [34]:
kubectl get sc

NAME         PROVISIONER                                     RECLAIMPOLICY   VOLUMEBINDINGMODE   ALLOWVOLUMEEXPANSION   AGE
nfs-client   cluster.local/nfs-subdir-external-provisioner   Delete          Immediate           true                   17s


In [35]:
kubectl patch storageclass nfs-client -p '{"metadata": {"annotations":{"storageclass.kubernetes.io/is-default-class":"true"}}}'

storageclass.storage.k8s.io/nfs-client patched


In [36]:
oc get sc -o yaml

apiVersion: v1
items:
- allowVolumeExpansion: true
  apiVersion: storage.k8s.io/v1
  kind: StorageClass
  metadata:
    annotations:
      meta.helm.sh/release-name: nfs-subdir-external-provisioner
      meta.helm.sh/release-namespace: nfs-provisioner
      storageclass.kubernetes.io/is-default-class: "true"
    creationTimestamp: "2022-06-11T18:28:44Z"
    labels:
      app: nfs-subdir-external-provisioner
      app.kubernetes.io/managed-by: Helm
      chart: nfs-subdir-external-provisioner-4.0.16
      heritage: Helm
      release: nfs-subdir-external-provisioner
    name: nfs-client
    resourceVersion: "34566"
    uid: 9d5f5829-4bb9-40ee-8346-c6735e74f5e7
  parameters:
    archiveOnDelete: "true"
  provisioner: cluster.local/nfs-subdir-external-provisioner
  reclaimPolicy: Delete
  volumeBindingMode: Immediate
kind: List
metadata:
  resourceVersion: ""
  selfLink: ""


In [37]:
oc new-project harbor

Now using project "harbor" on server "https://api.crc.testing:6443".

You can add applications to this project with the 'new-app' command. For example, try:

    oc new-app rails-postgresql-example

to build a new example application in Ruby. Or use kubectl to deploy a simple Kubernetes application:

    kubectl create deployment hello-node --image=k8s.gcr.io/e2e-test-images/agnhost:2.33 -- /agnhost serve-hostname



In [38]:
oc project harbor

Already on project "harbor" on server "https://api.crc.testing:6443".


Following https://github.com/goharbor/harbor-helm and https://computingforgeeks.com/install-harbor-image-registry-on-kubernetes-openshift-with-helm-chart/

In [39]:
helm repo add harbor https://helm.goharbor.io

"harbor" already exists with the same configuration, skipping


In [40]:
oc get all

No resources found in harbor namespace.


In [41]:
oc get events

No resources found in harbor namespace.


In [65]:
oc get sa

NAME       SECRETS   AGE
builder    2         44s
default    2         44s
deployer   2         44s


In [42]:
oc adm policy add-scc-to-user -z default anyuid

clusterrole.rbac.authorization.k8s.io/system:openshift:scc:anyuid added: "default"


In [43]:
helm ls

NAME	NAMESPACE	REVISION	UPDATED	STATUS	CHART	APP VERSION


In [44]:
export OPENSHIFT_DOMAIN=apps-crc.testing

In [45]:
helm repo update

Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "nfs-subdir-external-provisioner" chart repository
...Successfully got an update from the "csi-driver-smb" chart repository
...Successfully got an update from the "kong" chart repository
...Successfully got an update from the "confluentinc" chart repository
...Successfully got an update from the "dandydev" chart repository
...Successfully got an update from the "ingress-nginx" chart repository
...Successfully got an update from the "harbor" chart repository
...Successfully got an update from the "nginx-stable" chart repository
...Successfully got an update from the "sentry" chart repository
...Successfully got an update from the "elastic" chart repository
...Successfully got an update from the "jetstack" chart repository
...Successfully got an update from the "gitea-charts" chart repository
...Successfully got an update from the "argo" chart repository
...Successfully got an updat

In [46]:
helm upgrade --install harbor harbor/harbor \
--set persistence.persistentVolumeClaim.registry.accessMode=ReadWriteMany \
--set persistence.persistentVolumeClaim.registry.size=50Gi \
--set persistence.persistentVolumeClaim.chartmuseum.size=5Gi \
--set persistence.persistentVolumeClaim.database.size=5Gi \
--set externalURL=https://ocr.${OPENSHIFT_DOMAIN} \
--set expose.ingress.hosts.core=ocr.${OPENSHIFT_DOMAIN} \
--set expose.ingress.hosts.notary=notary.${OPENSHIFT_DOMAIN} \
--set harborAdminPassword=H@rb0rAdm \
-n harbor

Release "harbor" does not exist. Installing it now.
NAME: harbor
LAST DEPLOYED: Sat Jun 11 20:29:55 2022
NAMESPACE: harbor
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Please wait for several minutes for Harbor deployment to complete.
Then you should be able to visit the Harbor portal at https://ocr.apps-crc.testing
For more details, please visit https://github.com/goharbor/harbor


In [47]:
helm ls

NAME  	NAMESPACE	REVISION	UPDATED                                 	STATUS  	CHART       	APP VERSION
harbor	harbor   	1       	2022-06-11 20:29:55.310240909 +0200 CEST	deployed	harbor-1.9.1	2.5.1      


In [52]:
oc get all

NAME                                       READY   STATUS    RESTARTS   AGE
pod/harbor-chartmuseum-74bf669bf6-mkkpn    1/1     Running   0          39s
pod/harbor-core-7475b56c96-86jvj           0/1     Running   0          39s
pod/harbor-database-0                      1/1     Running   0          39s
pod/harbor-jobservice-7f7f464857-p7nkh     0/1     Running   0          39s
pod/harbor-notary-server-ffb8c7887-w2bqq   0/1     Running   0          39s
pod/harbor-notary-signer-59fbb6cc5-lc2c6   0/1     Running   0          39s
pod/harbor-portal-6bfb7684b5-lns8q         1/1     Running   0          39s
pod/harbor-redis-0                         1/1     Running   0          39s
pod/harbor-registry-6988cd44bf-tk6sn       2/2     Running   0          39s
pod/harbor-trivy-0                         1/1     Running   0          39s

NAME                           TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)             AGE
service/harbor-chartmuseum     ClusterIP   10.217.4.96    <none>   

In [53]:
oc get svc,ing,routes

NAME                           TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)             AGE
service/harbor-chartmuseum     ClusterIP   10.217.4.96    <none>        80/TCP              45s
service/harbor-core            ClusterIP   10.217.4.30    <none>        80/TCP              45s
service/harbor-database        ClusterIP   10.217.5.198   <none>        5432/TCP            45s
service/harbor-jobservice      ClusterIP   10.217.5.69    <none>        80/TCP              45s
service/harbor-notary-server   ClusterIP   10.217.5.196   <none>        4443/TCP            45s
service/harbor-notary-signer   ClusterIP   10.217.5.55    <none>        7899/TCP            45s
service/harbor-portal          ClusterIP   10.217.4.208   <none>        80/TCP              45s
service/harbor-redis           ClusterIP   10.217.4.234   <none>        6379/TCP            45s
service/harbor-registry        ClusterIP   10.217.4.91    <none>        5000/TCP,8080/TCP   45s
service/harbor-trivy           ClusterIP

In [83]:
helm uninstall my

Error: uninstall: Release not loaded: my: release: not found


: 1

In [55]:
oc get all

NAME                                       READY   STATUS    RESTARTS   AGE
pod/harbor-chartmuseum-74bf669bf6-mkkpn    1/1     Running   0          102s
pod/harbor-core-7475b56c96-86jvj           1/1     Running   0          102s
pod/harbor-database-0                      1/1     Running   0          102s
pod/harbor-jobservice-7f7f464857-p7nkh     1/1     Running   0          102s
pod/harbor-notary-server-ffb8c7887-w2bqq   1/1     Running   0          102s
pod/harbor-notary-signer-59fbb6cc5-lc2c6   1/1     Running   0          102s
pod/harbor-portal-6bfb7684b5-lns8q         1/1     Running   0          102s
pod/harbor-redis-0                         1/1     Running   0          102s
pod/harbor-registry-6988cd44bf-tk6sn       2/2     Running   0          102s
pod/harbor-trivy-0                         1/1     Running   0          102s

NAME                           TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)             AGE
service/harbor-chartmuseum     ClusterIP   10.217.4.96   

In [57]:
oc logs harbor-database-0 

ls: cannot access '/var/lib/postgresql/data/pgdata': No such file or directory
The files belonging to this database system will be owned by user "postgres".
This user must also own the server process.

The database cluster will be initialized with locales
  COLLATE:  en_US.UTF-8
  CTYPE:    en_US.UTF-8
  MESSAGES: C
  MONETARY: C
  NUMERIC:  C
  TIME:     C
The default text search configuration will be set to "english".

Data page checksums are disabled.

creating directory /var/lib/postgresql/data/pgdata/pg13 ... ok
creating subdirectories ... ok
selecting dynamic shared memory implementation ... posix
selecting default max_connections ... 100
selecting default shared_buffers ... 128MB
selecting default time zone ... UTC
creating configuration files ... ok
running bootstrap script ... ok
performing post-bootstrap initialization ... ok
syncing data to disk ... ok


Success. You can now start the database server using:

    pg_ctl -D /var/lib/postgresql/data/pgdata/pg13 -l logfile start

In [58]:
oc get pv

NAME                                       CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS      CLAIM                                                 STORAGECLASS   REASON   AGE
pv0001                                     100Gi      RWO,ROX,RWX    Recycle          Bound       openshift-image-registry/crc-image-registry-storage                           55d
pv0002                                     100Gi      RWO,ROX,RWX    Recycle          Available                                                                                 55d
pv0003                                     100Gi      RWO,ROX,RWX    Recycle          Available                                                                                 55d
pv0004                                     100Gi      RWO,ROX,RWX    Recycle          Available                                                                                 55d
pv0005                                     100Gi      RWO,ROX,RWX    Recycle          Available     

In [32]:
helm uninstall my

These resources were kept due to the resource policy:
[PersistentVolumeClaim] my-harbor-chartmuseum
[PersistentVolumeClaim] my-harbor-jobservice
[PersistentVolumeClaim] my-harbor-registry

release "my" uninstalled


In [35]:
oc get pvc

No resources found in harbor namespace.


In [34]:
oc delete pvc data-my-harbor-redis-0 data-my-harbor-trivy-0 database-data-my-harbor-database-0  my-harbor-chartmuseum my-harbor-jobservice my-harbor-registry    

persistentvolumeclaim "data-my-harbor-redis-0" deleted
persistentvolumeclaim "data-my-harbor-trivy-0" deleted
persistentvolumeclaim "database-data-my-harbor-database-0" deleted
persistentvolumeclaim "my-harbor-chartmuseum" deleted
persistentvolumeclaim "my-harbor-jobservice" deleted
persistentvolumeclaim "my-harbor-registry" deleted


In [40]:
oc get pv

NAME     CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS      CLAIM                                                 STORAGECLASS   REASON   AGE
pv0001   100Gi      RWO,ROX,RWX    Recycle          Bound       openshift-image-registry/crc-image-registry-storage                           15d
pv0002   100Gi      RWO,ROX,RWX    Recycle          Available                                                                                 15d
pv0004   100Gi      RWO,ROX,RWX    Recycle          Available                                                                                 15d
pv0005   100Gi      RWO,ROX,RWX    Recycle          Available                                                                                 15d
pv0006   100Gi      RWO,ROX,RWX    Recycle          Available                                                                                 15d
pv0007   100Gi      RWO,ROX,RWX    Recycle          Available                                                               

In [39]:
oc delete pv pv0003 pv0009 pv0015

persistentvolume "pv0003" deleted
persistentvolume "pv0009" deleted
persistentvolume "pv0015" deleted


In [41]:
helm install my harbor/harbor \
--set persistence.persistentVolumeClaim.registry.accessMode=ReadWriteMany \
--set persistence.persistentVolumeClaim.registry.size=50Gi \
--set persistence.persistentVolumeClaim.chartmuseum.size=5Gi \
--set persistence.persistentVolumeClaim.database.size=5Gi \
--set harborAdminPassword=H@rb0rAdm \
-n harbor

NAME: my
LAST DEPLOYED: Mon May  2 11:52:14 2022
NAMESPACE: harbor
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Please wait for several minutes for Harbor deployment to complete.
Then you should be able to visit the Harbor portal at https://core.harbor.domain
For more details, please visit https://github.com/goharbor/harbor


In [46]:
oc get all

NAME                                           READY   STATUS             RESTARTS      AGE
pod/my-harbor-chartmuseum-b579b7748-5s8gv      1/1     Running            0             41s
pod/my-harbor-core-f8565cf98-hzlhn             0/1     Running            0             41s
pod/my-harbor-database-0                       0/1     CrashLoopBackOff   2 (17s ago)   41s
pod/my-harbor-jobservice-5fd46cb686-lrklx      0/1     Running            0             41s
pod/my-harbor-notary-server-846567cd67-rkm42   0/1     Running            1 (8s ago)    41s
pod/my-harbor-notary-signer-6fb886b997-ts7s4   0/1     Running            1 (9s ago)    41s
pod/my-harbor-portal-5f5d5fbc77-g4mr2          1/1     Running            0             41s
pod/my-harbor-redis-0                          0/1     CrashLoopBackOff   2 (14s ago)   41s
pod/my-harbor-registry-76d6b6cb78-ltf2b        2/2     Running            0             41s
pod/my-harbor-trivy-0                          0/1     CrashLoopBackOff   2 (17s

In [47]:
oc logs my-harbor-database-0 

ls: cannot access '/var/lib/postgresql/data/pgdata': Permission denied
The files belonging to this database system will be owned by user "postgres".
This user must also own the server process.

The database cluster will be initialized with locales
  COLLATE:  en_US.UTF-8
  CTYPE:    en_US.UTF-8
  MESSAGES: C
  MONETARY: C
  NUMERIC:  C
  TIME:     C
The default text search configuration will be set to "english".

Data page checksums are disabled.

initdb: error: could not access directory "/var/lib/postgresql/data/pgdata/pg13": Permission denied


In [55]:
helm uninstall my

These resources were kept due to the resource policy:
[PersistentVolumeClaim] my-harbor-chartmuseum
[PersistentVolumeClaim] my-harbor-jobservice
[PersistentVolumeClaim] my-harbor-registry

release "my" uninstalled


In [58]:
oc get pvc -o jsonpath='{.items[*].metadata.name}'

data-my-harbor-redis-0 data-my-harbor-trivy-0 database-data-my-harbor-database-0 my-harbor-chartmuseum my-harbor-jobservice my-harbor-registry

In [59]:
oc delete pvc $(oc get pvc -o jsonpath='{.items[*].metadata.name}')

persistentvolumeclaim "data-my-harbor-redis-0" deleted
persistentvolumeclaim "data-my-harbor-trivy-0" deleted
persistentvolumeclaim "database-data-my-harbor-database-0" deleted
persistentvolumeclaim "my-harbor-chartmuseum" deleted
persistentvolumeclaim "my-harbor-jobservice" deleted
persistentvolumeclaim "my-harbor-registry" deleted


In [60]:
oc delete project harbor

project.project.openshift.io "harbor" deleted


In [3]:
oc get pv

NAME                                       CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS      CLAIM                                                 STORAGECLASS   REASON   AGE
pv0001                                     100Gi      RWO,ROX,RWX    Recycle          Bound       openshift-image-registry/crc-image-registry-storage                           16d
pv0002                                     100Gi      RWO,ROX,RWX    Recycle          Available                                                                                 16d
pv0004                                     100Gi      RWO,ROX,RWX    Recycle          Available                                                                                 16d
pv0005                                     100Gi      RWO,ROX,RWX    Recycle          Available                                                                                 16d
pv0006                                     100Gi      RWO,ROX,RWX    Recycle          Available     

In [4]:
oc get sc

NAME                   PROVISIONER                                     RECLAIMPOLICY   VOLUMEBINDINGMODE   ALLOWVOLUMEEXPANSION   AGE
nfs-client (default)   cluster.local/nfs-subdir-external-provisioner   Delete          Immediate           true                   20h


In [17]:
helm ls

NAME	NAMESPACE	REVISION	UPDATED                                 	STATUS  	CHART         	APP VERSION
my  	argocd   	1       	2022-05-05 14:35:00.509345717 +0200 CEST	deployed	argo-cd-3.29.1	v2.2.1     


In [18]:
oc project harbor

Now using project "harbor" on server "https://api.crc.testing:6443".


In [19]:
oc get all

NAME                                        READY   STATUS    RESTARTS       AGE
pod/harbor-chartmuseum-678fb8c649-4vt7l     1/1     Running   8              16d
pod/harbor-core-768dc66c9d-5z4fg            1/1     Running   14             16d
pod/harbor-database-0                       1/1     Running   9 (108m ago)   16d
pod/harbor-jobservice-d6f794797-fwhtg       1/1     Running   9              16d
pod/harbor-notary-server-557d494996-dr9hr   1/1     Running   28             16d
pod/harbor-notary-signer-5d9659f989-wn7c8   1/1     Running   26             16d
pod/harbor-portal-97fcbbd96-8g8sd           1/1     Running   9              16d
pod/harbor-redis-0                          1/1     Running   8              16d
pod/harbor-registry-5b9469b59-vvnhn         2/2     Running   16             16d
pod/harbor-trivy-0                          1/1     Running   8              16d

NAME                           TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)             AGE
service/harb

In [20]:
helm ls

NAME  	NAMESPACE	REVISION	UPDATED                                 	STATUS  	CHART       	APP VERSION
harbor	harbor   	3       	2022-05-02 13:55:36.506744432 +0200 CEST	deployed	harbor-1.9.0	2.5.0      


In [21]:
helm uninstall harbor

These resources were kept due to the resource policy:
[PersistentVolumeClaim] harbor-chartmuseum
[PersistentVolumeClaim] harbor-jobservice
[PersistentVolumeClaim] harbor-registry

release "harbor" uninstalled
